# Recommender System using AutoEncoder

### Data Preprocessing

In [1]:
# Importing the libraries
import numpy as np
import pandas as pd
from data_prepocess import load_data

DATASET = '../movielens-dataset/ratings.csv'

training_set, test_set, n_users, n_movies = load_data(DATASET)